In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

In [9]:
df = pd.read_csv('./Tabnet_Raw_final.csv')

In [10]:
df.replace((np.inf, -np.inf), np.nan, inplace=True)
df.dropna(inplace=True)

In [21]:
# With oversampled data by SMOTE using best hyperparameters set by optuna

train = df[df['회계년도'] <= 2017]
test = df[df['회계년도'] > 2017]

X_train = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).values
X_test = test.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).values

y_train = train['부실'].values
y_test = test['부실'].values

feature_names = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).columns.tolist()

# Resample using SMOTE
smote = SMOTE(sampling_strategy='minority' ,k_neighbors=5)
#smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
#X_resampled_test, y_resampled_test = smote.fit_resample(X_test, y_test)

# Resample using Random Under Sampler
#rus = RandomUnderSampler()
#X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
#X_resampled_test, y_resampled_test = rus.fit_resample(X_test, y_test)

X_train = X_resampled
X_test = X_test
y_train = y_resampled
y_test = y_test

feature_names = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).columns.tolist()

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=1)

clf = TabNetClassifier(
        n_d=46,
        n_a=38,
        n_steps=5,
        gamma=1.5515070542008145,
        lambda_sparse=0.5371050923936614,
        optimizer_params={"lr": 1e-3},
        verbose=0,
    )

    # Train TabNetClassifier on training data
clf.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_name=["val"],
        eval_metric=["auc"],
        max_epochs=10,
        patience=10,
        batch_size=1024,
        virtual_batch_size=128,
    )
preds = clf.predict(X_test)

cm = confusion_matrix(y_test, preds)
acc = accuracy_score(y_test, preds)
prec = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

print("Confusion matrix:\n", cm)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1-score:", f1)

# Feature importance
feat_importances = clf.feature_importances_
feature_importances_dict = dict(zip(feature_names, feat_importances))

# Print the feature importance scores
print(feature_importances_dict)

Stop training because you reached max_epochs = 10 with best_epoch = 9 and best_val_auc = 0.85924


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Confusion matrix:
 [[2171  334]
 [  33   54]]
Accuracy: 0.8584104938271605
Precision: 0.13917525773195877
Recall: 0.6206896551724138
F1-score: 0.22736842105263155
{'자기자본구성비율': 0.1328770561183189, '설비투자효율': 0.010130764364031646, '총자본투자효율': 0.03598956454915113, '이자보상배율(이자비용)': 0.050023970314183035, '유동비율': 0.01677674638018684, '당좌비율': 0.024929830033410188, '부채비율': 0.006045296825899569, '총자본정상영업이익률': 0.06008974342432949, '매출액정상영업이익률': 0.03181532866949353, '매출액순이익률': 0.020541046889065762, '자기자본순이익률': 0.041371586534249855, '매출채권회전률': 0.02005254839735838, '재고자산회전률': 0.022990908130517358, '총자본회전률': 0.00790391036018159, '순운전자본비율': 0.018525492996260714, '매출액증가율': 0.013424539667894269, '총자본증가율': 0.02744858606168681, '유동자산증가율': 0.003738112452669213, '유형자산증가율': 0.013073016755354843, '영업이익증가율': 0.010420399434713403, '순이익증가율': 0.012160735546197945, 'RETA': 0.032338742957855514, 'EBTA': 0.05996953713012594, 'OM': 0.018836360758296963, '종업원수증가율': 0.00808998338553831, '영업이익변화율': 0.004377510571429209, '

In [20]:
# Hyperparameter tuning with optuna

import optuna
import pandas as pd
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier

train = df[df['회계년도'] <= 2017]
test = df[df['회계년도'] > 2017]

X_train = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).values
X_test = test.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).values

y_train = train['부실'].values
y_test = test['부실'].values

feature_names = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).columns.tolist()

# Resample using SMOTE
smote = SMOTE(sampling_strategy='minority' ,k_neighbors=5)
#smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
#X_resampled_test, y_resampled_test = smote.fit_resample(X_test, y_test)

# Resample using Random Under Sampler
#rus = RandomUnderSampler()
#X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
#X_resampled_test, y_resampled_test = rus.fit_resample(X_test, y_test)

X_train = X_resampled
X_test = X_test
y_train = y_resampled
y_test = y_test

feature_names = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).columns.tolist()

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.2, random_state=1)


# Define objective function for Optuna
def objective(trial):
    # Define hyperparameters to tune
    n_d = trial.suggest_int("n_d", 8, 64)
    n_a = trial.suggest_int("n_a", 8, 64)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma = trial.suggest_float("gamma", 1.0, 2.0)
    lambda_sparse = trial.suggest_float("lambda_sparse", 0.0, 1.0)

    # Create TabNetClassifier with specified hyperparameters
    model = TabNetClassifier(
        n_d=n_d,46
        n_a=n_a,38
        n_steps=n_steps,5
        gamma=gamma,1.5515070542008145
        lambda_sparse=lambda_sparse,0.5371050923936614
        optimizer_params={"lr": 1e-3},
        verbose=0,
    )

    # Train TabNetClassifier on training data
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_name=["val"],
        eval_metric=["auc"],
        max_epochs=10,
        patience=10,
        batch_size=1024,
        virtual_batch_size=128,
    )

    # Return validation AUC score for Optuna to optimize
    return model.best_cost

# Run Optuna hyperparameter search
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print best hyperparameters and validation AUC score
print(f"Best params: {study.best_params}")
print(f"Best score: {study.best_value}")


[I 2023-04-27 20:48:12,967] A new study created in memory with name: no-name-477eb293-c7da-4641-949f-bdd2806e9fab


Stop training because you reached max_epochs = 10 with best_epoch = 9 and best_val_auc = 0.78966


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-04-27 20:48:38,195] Trial 0 finished with value: 0.7896553447609673 and parameters: {'n_d': 32, 'n_a': 52, 'n_steps': 4, 'gamma': 1.5434205762858304, 'lambda_sparse': 0.10732614367671434}. Best is trial 0 with value: 0.7896553447609673.


Stop training because you reached max_epochs = 10 with best_epoch = 9 and best_val_auc = 0.79843


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-04-27 20:49:06,051] Trial 1 finished with value: 0.7984267139048176 and parameters: {'n_d': 38, 'n_a': 35, 'n_steps': 5, 'gamma': 1.2543046223356065, 'lambda_sparse': 0.5021987924441852}. Best is trial 1 with value: 0.7984267139048176.


...

Stop training because you reached max_epochs = 10 with best_epoch = 8 and best_val_auc = 0.84977


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-04-27 21:42:17,356] Trial 98 finished with value: 0.8497748947041623 and parameters: {'n_d': 50, 'n_a': 44, 'n_steps': 3, 'gamma': 1.0025240918030596, 'lambda_sparse': 0.7965588799012722}. Best is trial 77 with value: 0.8564057450869953.


Stop training because you reached max_epochs = 10 with best_epoch = 9 and best_val_auc = 0.80222


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-04-27 21:42:40,810] Trial 99 finished with value: 0.8022235635840761 and parameters: {'n_d': 48, 'n_a': 43, 'n_steps': 3, 'gamma': 1.005490109532815, 'lambda_sparse': 0.8770051047905871}. Best is trial 77 with value: 0.8564057450869953.


Best params: {'n_d': 46, 'n_a': 38, 'n_steps': 5, 'gamma': 1.5515070542008145, 'lambda_sparse': 0.5371050923936614}
Best score: 0.8564057450869953


In [22]:
train = df[df['회계년도'] <= 2017]
test = df[df['회계년도'] > 2017]

X_train = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).values
X_test = test.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).values

y_train = train['부실'].values
y_test = test['부실'].values

feature_names = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).columns.tolist()

# Resample using SMOTE
#smote = SMOTE()
#X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
#X_resampled_test, y_resampled_test = smote.fit_resample(X_test, y_test)

sm = BorderlineSMOTE(random_state=42, sampling_strategy='minority')
X_res, y_res = sm.fit_resample(X_train, y_train)
X_res_test, y_res_test = sm.fit_resample(X_test, y_test)

# Resample using Random Under Sampler
#rus = RandomUnderSampler()
#X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
#X_resampled_test, y_resampled_test = rus.fit_resample(X_test, y_test)

X_train = X_res
X_test = X_res_test
y_train = y_res
y_test = y_res_test
#X_test = X_test
#y_test = y_test

feature_names = train.drop(['부실', '회사명', '회계년도', '거래소코드'], axis=1).columns.tolist()

X_train, X_valid, y_train, y_valid = train_test_split(
X_train, y_train, test_size=0.2, random_state=1)

clf = TabNetClassifier()
clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
preds = clf.predict(X_test)

cm = confusion_matrix(y_test, preds)
acc = accuracy_score(y_test, preds)
prec = precision_score(y_test, preds)
rec = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)

print("Confusion matrix:\n", cm)
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1-score:", f1)

# Feature importance
feat_importances = clf.feature_importances_
feature_importances_dict = dict(zip(feature_names, feat_importances))

# Print the feature importance scores
print(feature_importances_dict)

c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.64567 | val_0_auc: 0.75382 |  0:00:01s
epoch 1  | loss: 0.49475 | val_0_auc: 0.81611 |  0:00:02s
epoch 2  | loss: 0.43466 | val_0_auc: 0.86107 |  0:00:03s
epoch 3  | loss: 0.40824 | val_0_auc: 0.87643 |  0:00:04s
epoch 4  | loss: 0.39722 | val_0_auc: 0.88453 |  0:00:05s
epoch 5  | loss: 0.38396 | val_0_auc: 0.90632 |  0:00:06s
epoch 6  | loss: 0.37844 | val_0_auc: 0.90867 |  0:00:08s
epoch 7  | loss: 0.3671  | val_0_auc: 0.90816 |  0:00:09s
epoch 8  | loss: 0.36113 | val_0_auc: 0.90263 |  0:00:10s
epoch 9  | loss: 0.35198 | val_0_auc: 0.91918 |  0:00:11s
epoch 10 | loss: 0.33541 | val_0_auc: 0.92074 |  0:00:13s
epoch 11 | loss: 0.32865 | val_0_auc: 0.92368 |  0:00:14s
epoch 12 | loss: 0.32637 | val_0_auc: 0.92447 |  0:00:15s
epoch 13 | loss: 0.31584 | val_0_auc: 0.93141 |  0:00:16s
epoch 14 | loss: 0.31442 | val_0_auc: 0.93477 |  0:00:17s
epoch 15 | loss: 0.30716 | val_0_auc: 0.932   |  0:00:19s
epoch 16 | loss: 0.30122 | val_0_auc: 0.93274 |  0:00:20s
epoch 17 | los

c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Confusion matrix:
 [[2185  320]
 [ 292 2213]]
Accuracy: 0.8778443113772455
Precision: 0.8736675878405054
Recall: 0.883433133732535
F1-score: 0.8785232235013894
{'자기자본구성비율': 0.0004232720532696617, '설비투자효율': 8.561037829367222e-06, '총자본투자효율': 0.019527258715194947, '이자보상배율(이자비용)': 0.007741363144204045, '유동비율': 0.022316758739469, '당좌비율': 0.007314480000917542, '부채비율': 0.0, '총자본정상영업이익률': 0.00013020730101048503, '매출액정상영업이익률': 0.0, '매출액순이익률': 0.06638812713552099, '자기자본순이익률': 0.24002181042362317, '매출채권회전률': 0.00629503819036634, '재고자산회전률': 0.008553973453161345, '총자본회전률': 0.00015396005335002662, '순운전자본비율': 0.033507139539488105, '매출액증가율': 0.0016823719841356371, '총자본증가율': 0.01372220361576469, '유동자산증가율': 0.0, '유형자산증가율': 0.010147719009483993, '영업이익증가율': 0.007052290710246531, '순이익증가율': 0.0, 'RETA': 0.026332457285370325, 'EBTA': 0.03842151799980565, 'OM': 0.028228369816540585, '종업원수증가율': 1.0625171507134382e-06, '영업이익변화율': 0.0019375806828829214, '매출액변화율': 0.0003217300788560566, '당기순이익변화율': 0.029784311354

In [23]:
features = {'자기자본구성비율': 0.0004232720532696617, '설비투자효율': 8.561037829367222e-06, '총자본투자효율': 0.019527258715194947, 
            '이자보상배율(이자비용)': 0.007741363144204045, '유동비율': 0.022316758739469, '당좌비율': 0.007314480000917542, 
            '부채비율': 0.0, '총자본정상영업이익률': 0.00013020730101048503, '매출액정상영업이익률': 0.0, '매출액순이익률': 0.06638812713552099, 
            '자기자본순이익률': 0.24002181042362317, '매출채권회전률': 0.00629503819036634, '재고자산회전률': 0.008553973453161345, '총자본회전률': 0.00015396005335002662, 
            '순운전자본비율': 0.033507139539488105, '매출액증가율': 0.0016823719841356371, '총자본증가율': 0.01372220361576469, '유동자산증가율': 0.0, 
            '유형자산증가율': 0.010147719009483993, '영업이익증가율': 0.007052290710246531, '순이익증가율': 0.0, 'RETA': 0.026332457285370325, 
            'EBTA': 0.03842151799980565, 'OM': 0.028228369816540585, '종업원수증가율': 1.0625171507134382e-06, '영업이익변화율': 0.0019375806828829214, 
            '매출액변화율': 0.0003217300788560566, '당기순이익변화율': 0.02978431135494589, 'DOL': 0.046627726274766364, 'DFL': 0.015975578380821524, 
            'EV/EBITDA': 0.02229432701227696, '영업활동으로 인한 현금흐름': 0.00015280255713645582, '금융비용부담률': 0.1156496154107632, '고정비율': 0.0, 
            'R&D비율': 0.0, '채무부담비율': 1.2086855394997411e-05, '거래량회전율': 0.049089580992175585, '로그시가총액': 0.0, 
            '수정거래량': 0.04490539716986878, '거래량증가율': 0.0, '시가총액증가율': 0.015260593653775049, '시가총액': 0.1200187268504332}

my_feats = pd.Series(features)

feats = pd.DataFrame(my_feats)

feats.sort_values(by=0, ascending=False)

,0
자기자본순이익률,0.240022
시가총액,0.120019
금융비용부담률,0.115650
매출액순이익률,0.066388
거래량회전율,0.049090
DOL,0.046628
수정거래량,0.044905
EBTA,0.038422
순운전자본비율,0.033507
당기순이익변화율,0.029784
